# TODO
See CSV Tables
https://docutils.sourceforge.io/docs/ref/rst/directives.html#csv-table


In [6]:
from msticpy.data import QueryProvider
import pandas as pd
from msticpy.data import data_providers
from msticpy.data.data_providers import DataEnvironment


prov_list = ["AzureSentinel", "SecurityGraph", "MDATP", "LocalData", "Splunk"]
env_q_stores = {prov: QueryProvider(prov) for prov in prov_list}
    

query_series = []
for env, env_queries in env_q_stores.items():
    query_names = env_queries.list_queries()
    for query_name in query_names:
        q_group, q_name = query_name.split(".")
        query_path = [env, q_group, q_name]
        q_obj = getattr(getattr(env_queries, q_group), q_name)
        qry = env_queries.query_store.get_query(q_group + "." +  q_name)
        if "table" in qry.default_params:
           q_table = qry.default_params["table"].get("default", "na").split()[0]
        elif "table" in qry.required_params:
            q_table = qry.required_params["table"].get("default", "na").split()[0]
        else:
            q_table = "-"
        q_dict = {
            "Environment": env,
            "QueryGroup": q_group,
            "Query": q_name,
            "Description": qry.description,
            "ReqdParams": ", ".join([f"{param} ({p_data.get('type')})" for param, p_data in qry.required_params.items()]),
            #"OtherParams": ", ".join([f"{param}" for param in qry.default_params]),
            "Table": q_table
        }
        query_series.append(pd.Series(q_dict))

query_df = pd.DataFrame(query_series)
    
# env_queries.

from tabulate import tabulate

for name, grp in query_df.groupby("Environment"):
    print(f"Queries for {name}")
    print("-" * len(f"Queries for {name}"))
    print()
    print(tabulate(grp.drop(columns="Environment"), headers="keys", showindex="never", tablefmt="rst"))
    print("\n")

file_name = "../docs/source/data_aquisition/DataQueries.rst"
with open(file_name, "w") as rst_file:
    rst_file.write("Data Queries Reference\n")
    rst_file.write("=" * len("Data Queries Reference") + "\n\n")
    
    for name, grp in query_df.groupby("Environment"):
        rst_file.write(f"Queries for {name}\n")
        rst_file.write("-" * len(f"Queries for {name}") + "\n")
        rst_file.write("\n")
        tbl_txt = tabulate(grp.drop(columns="Environment"), headers="keys", showindex="never", tablefmt="rst")
        rst_file.write(tbl_txt)
        rst_file.write("\n\n")

     observables (list)                                                                                               ThreatIntelligenceIndicator
ThreatIntelligence  list_indicators_by_url            Retrieves list of indicators by URL                                                                          observables (list)                                                                                               ThreatIntelligenceIndicator
WindowsSecurity     get_host_logon                    Retrieves the logon event for the session id on the host                                                     start (datetime), end (datetime), host_name (str), logon_session_id (str)                                        SecurityEvent
WindowsSecurity     get_parent_process                Retrieves the parent process of a supplied process                                                           start (datetime), end (datetime), host_name (str), process_name (str), process_id (str), logon_se

FileNotFoundError: [Errno 2] No such file or directory: '../docs/source/data_aquisition/DataQueries.rst'

In [3]:
env_q_stores

{'LogAnalytics': <msticpy.data.data_providers.QueryProvider at 0x27824f28548>,
 'SecurityGraph': <msticpy.data.data_providers.QueryProvider at 0x27824f90048>,
 'MDATP': <msticpy.data.data_providers.QueryProvider at 0x2782501f848>}